<a href="https://colab.research.google.com/github/areias/bert_covid_sentiment/blob/main/bert_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data pre-processing

from https://github.com/digitalepidemiologylab/covid-twitter-bert/blob/master/preprocess/create_finetune_data.py

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
"""requirements 
tensorflow==2.2.0
tensorflow_addons==0.11.2
"""

'requirements \ntensorflow==2.2.0\ntensorflow_addons==0.11.2\n'

In [3]:
!pip install tensorflow==2.2.0

In [4]:
import tensorflow
print(tensorflow.__version__)

2.2.0


In [5]:
!pip install  tensorflow_addons==0.11.2


In [6]:
import tensorflow_addons
print(tensorflow_addons.__version__)

0.11.2


In [7]:
!git clone -b master https://github.com/digitalepidemiologylab/covid-twitter-bert.git 

fatal: destination path 'covid-twitter-bert' already exists and is not an empty directory.


In [8]:
!pip install transformers

In [10]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForSequenceClassification,
   AdamW,
   glue_convert_examples_to_features
)
import tensorflow as tf
import tensorflow_datasets as tfds
import json

In [11]:
import pandas as pd

In [12]:
df=pd.read_csv("/content/drive/MyDrive/train-4.csv")

In [13]:
df.head()

,Unnamed: 0,tweet_id,text,label
0,0,5.608318e+17,@GMA @GStephanopoulos #PalmDesert high school ...,0
1,1,7.720430e+17,SB121 [Passed] Meningococcal disease-pupils to...,0
2,2,5.262292e+17,@cabosetv @EvilGeniuses @EGiNcontroL @Razer th...,0
3,3,4.410094e+17,“@MizzTwerksum: All natural 😋🙌 squats not shot...,0
4,4,4.542856e+17,#travel #jobs Travel Immunization Nurse Specia...,0


In [14]:
df.label.value_counts()

 1    1134
-1    1134
 0    1134
Name: label, dtype: int64

In [15]:
# id,label,text
df=df.loc[:,["tweet_id","label", "text"]]
df.columns=['id','label', 'text']

In [16]:
import numpy as np

In [17]:
"""
    60% - train set,
    20% - dev/validation set,
    20% - test set"""

train, dev, test = np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [18]:
dev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 680 entries, 2111 to 1140
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      680 non-null    float64
 1   label   680 non-null    int64  
 2   text    680 non-null    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 21.2+ KB


In [19]:
import os 
#os.makedirs("covid-twitter-bert/data/finetune/originals/crowdbreaks")

In [20]:
train.to_csv("covid-twitter-bert/data/finetune/originals/crowdbreaks/train.tsv", sep="\t",index=False)
dev.to_csv("covid-twitter-bert/data/finetune/originals/crowdbreaks/dev.tsv", sep="\t",index=False)
test.to_csv("covid-twitter-bert/data/finetune/originals/crowdbreaks/test.tsv", sep="\t",index=False)


In [22]:
# args
"""
cd preprocess
python create_finetune_data.py \
  --run_prefix test_run \
  --finetune_datasets <dataset_name> \
  --model_class bert_large_uncased_wwm \
  --max_seq_length 96 \
  --asciify_emojis \
  --username_filler twitteruser \
  --url_filler twitterurl \
  --replace_multiple_usernames \
  --replace_multiple_urls \
  --remove_unicode_symbols"
"""

'\ncd preprocess\npython create_finetune_data.py   --run_prefix test_run   --finetune_datasets <dataset_name>   --model_class bert_large_uncased_wwm   --max_seq_length 96   --asciify_emojis   --username_filler twitteruser   --url_filler twitterurl   --replace_multiple_usernames   --replace_multiple_urls   --remove_unicode_symbols"\n'

In [23]:
from collections import namedtuple
arguments = namedtuple('arguments', ['run_prefix','finetune_datasets','model_class',
                                     'max_seq_length', 'asciify_emojis','username_filler',
                                    'url_filler', 'replace_multiple_usernames','replace_multiple_urls',
                                      'remove_unicode_symbols','replace_usernames','replace_urls',
                                     'standardize_punctuation','remove_accented_characters'])

args = arguments("test_run",["crowdbreaks"],"covid-twitter-bert-2",
                 96, True, "twitteruser", 
                 "twitterurl", True,True,
                 True, True, True,
                 True, True)
args

arguments(run_prefix='test_run', finetune_datasets=['crowdbreaks'], model_class='covid-twitter-bert-2', max_seq_length=96, asciify_emojis=True, username_filler='twitteruser', url_filler='twitterurl', replace_multiple_usernames=True, replace_multiple_urls=True, remove_unicode_symbols=True, replace_usernames=True, replace_urls=True, standardize_punctuation=True, remove_accented_characters=True)

In [24]:
REQUIRED_COLUMNS = ['id', 'label', 'text']
DATA_DIR = os.path.join('covid-twitter-bert/data')
VOCAB_PATH = os.path.join('covid-twitter-bert/vocabs')


In [25]:
import datetime
def get_run_name(args):
    # Use timestamp to generate a unique run name
    ts = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S_%f')
    if args.run_prefix:
        run_name = f'run_{ts}_{args.run_prefix}'
    else:
        run_name = f'run_{ts}'
    return run_name

In [26]:
run_name = get_run_name(args)
run_name

'run_2021-11-18_15-24-57_560898_test_run'

In [27]:
run_dir = os.path.join(DATA_DIR, 'finetune', run_name)
run_dir

'covid-twitter-bert/data/finetune/run_2021-11-18_15-24-57_560898_test_run'

In [28]:
if not os.path.isdir(run_dir):
  os.makedirs(run_dir)

In [29]:
# find input data
originals_dir = os.path.join(DATA_DIR, 'finetune', 'originals')
originals_dir   

'covid-twitter-bert/data/finetune/originals'

In [30]:
if args.finetune_datasets is None or len(args.finetune_datasets) == 0:
    finetune_datasets = os.listdir(originals_dir)
else:
    finetune_datasets = args.finetune_datasets
finetune_datasets

['crowdbreaks']

In [31]:
# Pretrained models configuration, add model configuration here

PRETRAINED_MODELS = {
        'bert_large_uncased': {
            'bucket_location': 'pretrained_models/bert/keras_bert/uncased_L-24_H-1024_A-16',
            'hub_url': 'tensorflow/bert_en_uncased_L-24_H-1024_A-16/2',
            'config': 'bert_config_large_uncased.json',
            'is_tfhub_model': True,
            'vocab_file': 'bert-large-uncased-vocab.txt',
            'lower_case': True,
            'do_whole_word_masking': False
            },
        'bert_multi_cased': {
            'bucket_location': 'pretrained_models/bert/keras_bert/multi_cased_L-12_H-768_A-12',
            'hub_url': 'tensorflow/bert_multi_cased_L-12_H-768_A-12/2',
            'config': 'bert_config_multi_cased.json',
            'is_tfhub_model': True,
            'vocab_file': 'bert-multi-cased-vocab.txt',
            'lower_case': False,
            'do_whole_word_masking': False
            },
        'bert_large_uncased_wwm': {
            'bucket_location': 'pretrained_models/bert/keras_bert/wwm_uncased_L-24_H-1024_A-16',
            'hub_url': 'tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/2',
            'config': 'bert_config_large_uncased_wwm.json',
            'is_tfhub_model': True,
            'vocab_file': 'bert-large-uncased-whole-word-masking-vocab.txt',
            'lower_case': True,
            'do_whole_word_masking': True
            },
        'covid-twitter-bert': {
            'hub_url': 'digitalepidemiologylab/covid-twitter-bert/1',
            'is_tfhub_model': True,
            'config': 'bert_config_covid_twitter_bert.json',
            'vocab_file': 'bert-large-uncased-whole-word-masking-vocab.txt',
            'lower_case': True,
            'do_whole_word_masking': True
            },
        'covid-twitter-bert-2': {
            'hub_url': 'digitalepidemiologylab/covid-twitter-bert/2',
            'is_tfhub_model': True,
            'config': 'bert_config_covid_twitter_bert.json',
            'vocab_file': 'bert-large-uncased-whole-word-masking-vocab.txt',
            'lower_case': True,
            'do_whole_word_masking': True
            }
        }

In [32]:
# https://tfhub.dev/digitalepidemiologylab/covid-twitter-bert/2




In [33]:
do_lower_case = PRETRAINED_MODELS[args.model_class]['lower_case']
do_lower_case


True

In [34]:
# compile regexes
import re
from html.parser import HTMLParser
import unicodedata

username_regex = re.compile(r'(^|[^@\w])@(\w{1,15})\b')
url_regex = re.compile(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))')
control_char_regex = re.compile(r'[\r\n\t]+')
# translate table for punctuation
transl_table = dict([(ord(x), ord(y)) for x, y in zip(u"‘’´“”–-",  u"'''\"\"--")])
# HTML parser
html_parser = HTMLParser()

In [35]:
def standardize_text(text):
    """
    1) Escape HTML
    2) Replaces some non-standard punctuation with standard versions. 
    3) Replace \r, \n and \t with white spaces
    4) Removes all other control characters and the NULL byte
    5) Removes duplicate white spaces
    """
    html_parser = HTMLParser()

    # escape HTML symbols
    text = html_parser.unescape(text)
    # standardize punctuation
    text = text.translate(transl_table)
    text = text.replace('…', '...')
    # replace \t, \n and \r characters by a whitespace
    text = re.sub(control_char_regex, ' ', text)
    # remove all remaining control characters
    text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')
    # replace multiple spaces with single space
    text = ' '.join(text.split())
    return text.strip()

In [36]:
def replace_usernames(text, filler='user'):
    # @<user> is a marker used internally. use filler instead
    text = text.replace('@<user>', f'{filler}')
    # replace other user handles by filler
    text = re.sub(username_regex, filler, text)
    # add spaces between, and remove double spaces again
    text = text.replace(filler, f' {filler} ')
    text = ' '.join(text.split())
    return text

In [37]:
def replace_urls(text, filler='url'):
    # <url> is a marker used internally. use filler instead
    text = text.replace('<url>', filler)
    # replace other urls by filler
    text = re.sub(url_regex, filler, text)
    # add spaces between, and remove double spaces again
    text = text.replace(filler, f' {filler} ')
    text = ' '.join(text.split())
    return text

In [38]:
!pip install emoji
import emoji
def asciify_emojis(text):
    """
    Converts emojis into text aliases. E.g. 👍 becomes :thumbs_up:
    For a full list of text aliases see: https://www.webfx.com/tools/emoji-cheat-sheet/
    """
    text = emoji.demojize(text)
    return text

In [39]:
!pip install unidecode 
import unidecode
def standardize_punctuation(text):
    return ''.join([unidecode.unidecode(t) if unicodedata.category(t)[0] == 'P' else t for t in text])


In [40]:

def replace_multi_occurrences(text, filler):
    """Replaces multiple occurrences of filler with n filler"""
    # only run if we have multiple occurrences of filler
    if text.count(filler) <= 1:
        return text
    # pad fillers with whitespace
    text = text.replace(f'{filler}', f' {filler} ')
    # remove introduced duplicate whitespaces
    text = ' '.join(text.split())
    # find indices of occurrences
    indices = []
    for m in re.finditer(r'{}'.format(filler), text):
        index = m.start()
        indices.append(index)
    # collect merge list
    merge_list = []
    for i, index in enumerate(indices):
        if i > 0 and index - old_index == len(filler) + 1:
            # found two consecutive fillers
            if len(merge_list) > 0 and merge_list[-1][1] == old_index:
                # extend previous item
                merge_list[-1][1] = index
                merge_list[-1][2] += 1
            else:
                # create new item
                merge_list.append([old_index, index, 2])
        old_index = index
    # merge occurrences
    if len(merge_list) > 0:
        new_text = ''
        pos = 0
        for (start, end, count) in merge_list:
            new_text += text[pos:start]
            new_text += f'{count} {filler}'
            pos = end + len(filler)
        new_text += text[pos:]
        text = new_text
    return text

In [41]:
def remove_unicode_symbols(text):
    text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'So')
    return text


In [42]:
def remove_accented_characters(text):
    text = unidecode.unidecode(text)
    return text

In [43]:
# copying old commit of tensorflow models they used
#!cp -r /content/drive/MyDrive/models-93490036e00f37ecbe6693b9ff4ae488bb8e9270/* covid-twitter-bert/tensorflow_models/

In [44]:
import sys
sys.path.append('covid-twitter-bert/tensorflow_models')

In [45]:
!pip install sentencepiece

In [46]:
import importlib
from official.nlp.data.classifier_data_lib import DataProcessor, generate_tf_record_from_data_file, InputExample
from official.nlp.bert import tokenization


In [47]:
?importlib.reload

In [48]:
def preprocess_bert(text, args, do_lower_case=True):
    """Preprocesses tweet for BERT"""
    # standardize
    text = standardize_text(text)
    # replace usernames/urls
    if args.replace_usernames:
        text = replace_usernames(text, filler=args.username_filler)
    if args.replace_urls:
        text = replace_urls(text, filler=args.url_filler)
    if args.asciify_emojis:
        text = asciify_emojis(text)
    if args.standardize_punctuation:
        text = standardize_punctuation(text)
    if do_lower_case:
        text = text.lower()
    if args.replace_multiple_usernames:
        text = replace_multi_occurrences(text, args.username_filler)
    if args.replace_multiple_urls:
        text = replace_multi_occurrences(text, args.url_filler)
    if args.remove_unicode_symbols:
        text = remove_unicode_symbols(text)
    if args.remove_accented_characters:
        text = remove_accented_characters(text)
    return text

In [49]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)-5.5s] [%(name)-12.12s]: %(message)s')


In [50]:
def get_tokenizer(model_class):
    model = PRETRAINED_MODELS[model_class]
    vocab_file = os.path.join(VOCAB_PATH, model['vocab_file'])
    tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=model['lower_case'])
    return tokenizer

In [51]:
def generate_tfrecords(args, dataset_dir, labels):
    """Generates tfrecords from generated tsv files"""
    processor = TextClassificationProcessor(labels)
    # save label mapping
    processor.save_label_mapping(dataset_dir)
    # get tokenizer
    tokenizer = get_tokenizer(args.model_class)
    processor_text_fn = tokenization.convert_to_unicode
    # generate tfrecords
    input_dir = os.path.join(dataset_dir, 'preprocessed')
    output_dir = os.path.join(dataset_dir, 'tfrecords')
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
    input_meta_data = generate_tf_record_from_data_file(
        processor,
        input_dir,
        tokenizer,
        train_data_output_path=os.path.join(output_dir, 'train.tfrecords'),
        eval_data_output_path=os.path.join(output_dir, 'dev.tfrecords'),
        max_seq_length=args.max_seq_length)
    with tf.io.gfile.GFile(os.path.join(dataset_dir, 'meta.json'), 'w') as writer:
        writer.write(json.dumps(input_meta_data, indent=4) + '\n')
    logger.info(f'Sucessfully wrote tfrecord files to {output_dir}')


In [52]:
class TextClassificationProcessor(DataProcessor):
    """Processor for arbitrary text classification data"""

    def __init__(self, labels):
        self.labels = labels

    def save_label_mapping(self, data_dir):
        with open(os.path.join(data_dir, 'label_mapping.json'), 'w') as f:
            json.dump(self.labels, f)

    def get_examples(self, data_dir, _type):
        f_path = os.path.join(data_dir, f'{_type}.tsv')
        lines = self._read_tsv(f_path)
        return self._create_examples(lines, _type)

    def get_train_examples(self, data_dir):
        return self.get_examples(data_dir, 'train')

    def get_dev_examples(self, data_dir):
        return self.get_examples(data_dir, 'dev')

    def get_test_examples(self, data_dir):
        return self.get_examples(data_dir, 'test')

    def get_labels(self):
        return self.labels

    @staticmethod
    def get_processor_name():
        return 'text-classification'

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for i, line in enumerate(lines):
            guid = f'{set_type}-{i}'
            text_a = tokenization.convert_to_unicode(line[REQUIRED_COLUMNS.index('text')])
            if set_type == 'test':
                label = '0'
            else:
                label = tokenization.convert_to_unicode(line[REQUIRED_COLUMNS.index('label')])
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

In [53]:
# only works when changing line 462 in 
# official.nlp.data.classifier_data_lib.py to  
# label_id = label_map[int(example.label)]


# bert uses wordpiece token which explains why surfboard is tokenised as surf ##board
# https://towardsdatascience.com/how-to-build-a-wordpiece-tokenizer-for-bert-f505d97dddbb
or dataset in finetune_datasets:
    logger.info(f'Processing dataset {dataset}...')
    preprocessed_folder = os.path.join(run_dir, dataset, 'preprocessed')
    if not os.path.isdir(preprocessed_folder):
        os.makedirs(preprocessed_folder)
    labels = set()
    for _type in ['train', 'dev']:
        f_name = f'{_type}.tsv'
        logger.info(f'Reading data for for type {_type}...')
        f_path = os.path.join(originals_dir, dataset, f_name)
        if not os.path.isfile(f_path):
            logger.info(f'Could not find file {f_path}. Skipping.')
            continue
        df = pd.read_csv(f_path, usecols=REQUIRED_COLUMNS, sep='\t')
        logger.info('Creating preprocessed files...')
        df.loc[:, 'text'] = df.text.apply(preprocess_bert, args=(args, do_lower_case))
        df.to_csv(os.path.join(preprocessed_folder, f_name), columns=REQUIRED_COLUMNS, header=False, index=False, sep='\t')
        # add labels
        labels.update(df.label.unique().tolist())
    logger.info('Creating tfrecords files...')
    # we sort the labels alphabetically in order to maintain consistent label ids
    labels = sorted(list(labels))
    dataset_dir = os.path.join(run_dir, dataset)
    generate_tfrecords(args, dataset_dir, labels)

2021-11-18 15:25:07,820 [INFO ] [__main__    ]: Processing dataset crowdbreaks...
2021-11-18 15:25:07,823 [INFO ] [__main__    ]: Reading data for for type train...
2021-11-18 15:25:07,841 [INFO ] [__main__    ]: Creating preprocessed files...
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  if sys.path[0] == '':
2021-11-18 15:25:09,830 [INFO ] [__main__    ]: Reading data for for type dev...
2021-11-18 15:25:09,838 [INFO ] [__main__    ]: Creating preprocessed files...
2021-11-18 15:25:10,455 [INFO ] [__main__    ]: Creating tfrecords files...
2021-11-18 15:25:10,575 [INFO ] [absl        ]: Writing example 0 of 2041
2021-11-18 15:25:10,580 [INFO ] [absl        ]: *** Example ***
2021-11-18 15:25:10,582 [INFO ] [absl        ]: guid: train-0
2021-11-18 15:25:10,588 [INFO ] [absl        ]: tokens: [CLS] arizona classrooms vulnerable to me ##as ##les outbreak : 

In [59]:

class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [60]:
def save_to_json(data, f_name):
    with tf.io.gfile.GFile(f_name, 'w') as writer:
        writer.write(json.dumps(data, cls=JSONEncoder, indent=4))

In [61]:
f_path_config = os.path.join(run_dir, 'create_finetune_config.json')
logger.info(f'Saving config to {f_path_config}')
save_to_json(args, f_path_config)

2021-11-18 15:30:06,565 [INFO ] [__main__    ]: Saving config to covid-twitter-bert/data/finetune/run_2021-11-18_15-24-57_560898_test_run/create_finetune_config.json


## Training

from https://github.com/digitalepidemiologylab/covid-twitter-bert/blob/c87912b409659f40018e839c4124be5ae2486713/run_finetune.py


In [70]:
%%shell
python covid-twitter-bert/run_finetune.py

Traceback (most recent call last):
  File "covid-twitter-bert/run_finetune.py", line 4, in <module>
    from official.nlp.bert import bert_models
ModuleNotFoundError: No module named 'official'


CalledProcessError: ignored